**1. AntColony Optimization using Squared Sine Activation function**

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import numpy as np
# Loading MNIST data set
mnist = tf.keras.datasets.mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Normalizing the pixel values to the range [0, 1]
X_train, X_test = X_train / 255.0, X_test / 255.0

# Converting labels to categorical
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
class SquaredSineActivation(tf.keras.layers.Layer):
    def __init__(self, omega=1.0, **kwargs):
        super(SquaredSineActivation, self).__init__(**kwargs)
        self.omega = omega

    def call(self, inputs):
        return tf.sin(self.omega * inputs) ** 2

# CNN architecture with Squared Sine Activation
def create_cnn_model(activation_function):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same', activation=activation_function, input_shape=(28, 28, 1)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation=activation_function))
    model.add(Dense(10, activation='softmax'))
    return model
# Compiling the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# Define the objective function for Ant Colony Optimization
def objective_function(params):
    learning_rate = params[0]
    batch_size = int(params[1])
    epochs = int(params[2])

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    history = model.fit(X_train, y_train, validation_data=(X_test, y_test),
                        batch_size=batch_size, epochs=epochs, verbose=0)

    train_loss = history.history['loss'][-1]
    train_accuracy = history.history['accuracy'][-1]

    return train_loss + 1 - train_accuracy  # ACO minimizes the objective function, so maximize accuracy.

# Define the search space for ACO
param_boundaries = [
    (1e-6, 1e-2),  # Learning rate
    (32, 128),     # Batch size
    (5, 50)        # Epochs
]
# Number of ants in the colony
num_ants = 10
# Number of iterations
num_iterations = 50
# Evaporation rate
evaporation_rate = 0.5
# Pheromone decay rate
pheromone_decay = 0.1
class AntColonyOptimizer:
    def __init__(self, num_ants, objective_function, param_boundaries, evaporation_rate, pheromone_decay):
        self.num_ants = num_ants
        self.objective_function = objective_function
        self.param_boundaries = param_boundaries
        self.evaporation_rate = evaporation_rate
        self.pheromone_decay = pheromone_decay
        self.pheromone_levels = np.ones(len(param_boundaries[0]))

    def optimize(self, num_iterations):
        # Initialization
        best_params = None
        best_fitness = float('inf')

        for iteration in range(num_iterations):
            for ant in range(self.num_ants):
                params = [self._get_ant_param(boundaries) for boundaries in self.param_boundaries]
                fitness = self.objective_function(params)

                # Update global best
                if fitness < best_fitness:
                    best_params = params
                    best_fitness = fitness

                # Update pheromone levels
                self.pheromone_levels *= (1 - self.pheromone_decay)
                self.pheromone_levels += (1 / (fitness + 1))  # Fitness should be positive, avoid division by zero

            # Evaporation
            self.pheromone_levels *= self.evaporation_rate

        return best_params

    def _get_ant_param(self, boundaries):
        probabilities = self.pheromone_levels / np.sum(self.pheromone_levels)
        selected_index = np.random.choice(len(boundaries), p=probabilities)
        return np.random.uniform(*boundaries[selected_index])
# Creating an Ant Colony Optimizer object
aco = AntColonyOptimizer(num_ants, objective_function, param_boundaries, evaporation_rate, pheromone_decay)
# Finding the optimal parameters
optimal_params = aco.optimize(num_iterations)
# Displaying the best parameters found
print(f"Best parameters found by ACO: {optimal_params}")

# Compiling the model with the best parameters
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=optimal_params[0]),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Training the model with the best parameters
history = model.fit(X_train, y_train, batch_size=int(optimal_params[1]), epochs=int(optimal_params[2]), verbose=1)

# Evaluating the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test loss: {test_loss}, Test accuracy: {test_accuracy}")

**2. AntColony Optimization using relu Activation function**

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
# Loading MNIST data set
mnist = tf.keras.datasets.mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
# Normalizing the pixel values to the range [0, 1]
X_train, X_test = X_train / 255.0, X_test / 255.0
# Converting labels to categorical
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
# Converting labels to categorical
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
# CNN architecture
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation='softmax'))
# Compiling the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# Define the objective function for Ant Colony Optimization
def objective_function(params):
    learning_rate = params[0]
    batch_size = int(params[1])
    epochs = int(params[2])

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    history = model.fit(X_train, y_train, validation_data=(X_test, y_test),
                        batch_size=batch_size, epochs=epochs, verbose=0)

    train_loss = history.history['loss'][-1]
    train_accuracy = history.history['accuracy'][-1]

    return train_loss + 1 - train_accuracy  # ACO minimizes the objective function, so maximize accuracy.

# Define the search space for ACO
param_boundaries = [
    (1e-6, 1e-2),  # Learning rate
    (32, 128),     # Batch size
    (5, 50)        # Epochs
]
# Number of ants in the colony
num_ants = 10
# Number of iterations
num_iterations = 50
# Evaporation rate
evaporation_rate = 0.5
# Pheromone decay rate
pheromone_decay = 0.1
class AntColonyOptimizer:
    def __init__(self, num_ants, objective_function, param_boundaries, evaporation_rate, pheromone_decay):
        self.num_ants = num_ants
        self.objective_function = objective_function
        self.param_boundaries = param_boundaries
        self.evaporation_rate = evaporation_rate
        self.pheromone_decay = pheromone_decay
        self.pheromone_levels = np.ones(len(param_boundaries[0]))

    def optimize(self, num_iterations):
        # Initialization
        best_params = None
        best_fitness = float('inf')

        for iteration in range(num_iterations):
            for ant in range(self.num_ants):
                params = [self._get_ant_param(boundaries) for boundaries in self.param_boundaries]
                fitness = self.objective_function(params)

                # Update global best
                if fitness < best_fitness:
                    best_params = params
                    best_fitness = fitness

                # Update pheromone levels
                self.pheromone_levels *= (1 - self.pheromone_decay)
                self.pheromone_levels += (1 / (fitness + 1))  # Fitness should be positive, avoid division by zero

            # Evaporation
            self.pheromone_levels *= self.evaporation_rate

        return best_params

    def _get_ant_param(self, boundaries):
        probabilities = self.pheromone_levels / np.sum(self.pheromone_levels)
        selected_index = np.random.choice(len(boundaries), p=probabilities)
        return np.random.uniform(*boundaries[selected_index])
# Creating an Ant Colony Optimizer object
aco = AntColonyOptimizer(num_ants, objective_function, param_boundaries, evaporation_rate, pheromone_decay)
# Finding the optimal parameters
optimal_params = aco.optimize(num_iterations)
# Displaying the best parameters found
print(f"Best parameters found by ACO: {optimal_params}")

# Compiling the model with the best parameters
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=optimal_params[0]),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Training the model with the best parameters
history = model.fit(X_train, y_train, batch_size=int(optimal_params[1]), epochs=int(optimal_params[2]), verbose=1)

# Evaluating the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test loss: {test_loss}, Test accuracy: {test_accuracy}")

**3. AntColony Optimization using Sigmoid Activation function**

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
# Loading MNIST data set
mnist = tf.keras.datasets.mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
# Normalizing the pixel values to the range [0, 1]
X_train, X_test = X_train / 255.0, X_test / 255.0
# Converting labels to categorical
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
# Converting labels to categorical
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
# CNN architecture
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', activation='sigmoid', input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='sigmoid'))
model.add(Dense(10, activation='softmax'))
# Compiling the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# Define the objective function for Ant Colony Optimization
def objective_function(params):
    learning_rate = params[0]
    batch_size = int(params[1])
    epochs = int(params[2])

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    history = model.fit(X_train, y_train, validation_data=(X_test, y_test),
                        batch_size=batch_size, epochs=epochs, verbose=0)

    train_loss = history.history['loss'][-1]
    train_accuracy = history.history['accuracy'][-1]

    return train_loss + 1 - train_accuracy  # ACO minimizes the objective function, so maximize accuracy.

# Define the search space for ACO
param_boundaries = [
    (1e-6, 1e-2),  # Learning rate
    (32, 128),     # Batch size
    (5, 50)        # Epochs
]
# Number of ants in the colony
num_ants = 10
# Number of iterations
num_iterations = 50
# Evaporation rate
evaporation_rate = 0.5
# Pheromone decay rate
pheromone_decay = 0.1
class AntColonyOptimizer:
    def __init__(self, num_ants, objective_function, param_boundaries, evaporation_rate, pheromone_decay):
        self.num_ants = num_ants
        self.objective_function = objective_function
        self.param_boundaries = param_boundaries
        self.evaporation_rate = evaporation_rate
        self.pheromone_decay = pheromone_decay
        self.pheromone_levels = np.ones(len(param_boundaries[0]))

    def optimize(self, num_iterations):
        # Initialization
        best_params = None
        best_fitness = float('inf')

        for iteration in range(num_iterations):
            for ant in range(self.num_ants):
                params = [self._get_ant_param(boundaries) for boundaries in self.param_boundaries]
                fitness = self.objective_function(params)

                # Update global best
                if fitness < best_fitness:
                    best_params = params
                    best_fitness = fitness

                # Update pheromone levels
                self.pheromone_levels *= (1 - self.pheromone_decay)
                self.pheromone_levels += (1 / (fitness + 1))  # Fitness should be positive, avoid division by zero

            # Evaporation
            self.pheromone_levels *= self.evaporation_rate

        return best_params

    def _get_ant_param(self, boundaries):
        probabilities = self.pheromone_levels / np.sum(self.pheromone_levels)
        selected_index = np.random.choice(len(boundaries), p=probabilities)
        return np.random.uniform(*boundaries[selected_index])
# Creating an Ant Colony Optimizer object
aco = AntColonyOptimizer(num_ants, objective_function, param_boundaries, evaporation_rate, pheromone_decay)
# Finding the optimal parameters
optimal_params = aco.optimize(num_iterations)
# Displaying the best parameters found
print(f"Best parameters found by ACO: {optimal_params}")

# Compiling the model with the best parameters
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=optimal_params[0]),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Training the model with the best parameters
history = model.fit(X_train, y_train, batch_size=int(optimal_params[1]), epochs=int(optimal_params[2]), verbose=1)

# Evaluating the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test loss: {test_loss}, Test accuracy: {test_accuracy}")

**4. AntColony Optimization using tanh Activation function**

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
# Loading MNIST data set
mnist = tf.keras.datasets.mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
# Normalizing the pixel values to the range [0, 1]
X_train, X_test = X_train / 255.0, X_test / 255.0
# Converting labels to categorical
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
# Converting labels to categorical
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
# CNN architecture
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', activation='tanh', input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='tanh'))
model.add(Dense(10, activation='softmax'))
# Compiling the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# Define the objective function for Ant Colony Optimization
def objective_function(params):
    learning_rate = params[0]
    batch_size = int(params[1])
    epochs = int(params[2])

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    history = model.fit(X_train, y_train, validation_data=(X_test, y_test),
                        batch_size=batch_size, epochs=epochs, verbose=0)

    train_loss = history.history['loss'][-1]
    train_accuracy = history.history['accuracy'][-1]

    return train_loss + 1 - train_accuracy  # ACO minimizes the objective function, so maximize accuracy.

# Define the search space for ACO
param_boundaries = [
    (1e-6, 1e-2),  # Learning rate
    (32, 128),     # Batch size
    (5, 50)        # Epochs
]
# Number of ants in the colony
num_ants = 10
# Number of iterations
num_iterations = 50
# Evaporation rate
evaporation_rate = 0.5
# Pheromone decay rate
pheromone_decay = 0.1
class AntColonyOptimizer:
    def __init__(self, num_ants, objective_function, param_boundaries, evaporation_rate, pheromone_decay):
        self.num_ants = num_ants
        self.objective_function = objective_function
        self.param_boundaries = param_boundaries
        self.evaporation_rate = evaporation_rate
        self.pheromone_decay = pheromone_decay
        self.pheromone_levels = np.ones(len(param_boundaries[0]))

    def optimize(self, num_iterations):
        # Initialization
        best_params = None
        best_fitness = float('inf')

        for iteration in range(num_iterations):
            for ant in range(self.num_ants):
                params = [self._get_ant_param(boundaries) for boundaries in self.param_boundaries]
                fitness = self.objective_function(params)

                # Update global best
                if fitness < best_fitness:
                    best_params = params
                    best_fitness = fitness

                # Update pheromone levels
                self.pheromone_levels *= (1 - self.pheromone_decay)
                self.pheromone_levels += (1 / (fitness + 1))  # Fitness should be positive, avoid division by zero

            # Evaporation
            self.pheromone_levels *= self.evaporation_rate

        return best_params

    def _get_ant_param(self, boundaries):
        probabilities = self.pheromone_levels / np.sum(self.pheromone_levels)
        selected_index = np.random.choice(len(boundaries), p=probabilities)
        return np.random.uniform(*boundaries[selected_index])
# Creating an Ant Colony Optimizer object
aco = AntColonyOptimizer(num_ants, objective_function, param_boundaries, evaporation_rate, pheromone_decay)
# Finding the optimal parameters
optimal_params = aco.optimize(num_iterations)
# Displaying the best parameters found
print(f"Best parameters found by ACO: {optimal_params}")

# Compiling the model with the best parameters
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=optimal_params[0]),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Training the model with the best parameters
history = model.fit(X_train, y_train, batch_size=int(optimal_params[1]), epochs=int(optimal_params[2]), verbose=1)

# Evaluating the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test loss: {test_loss}, Test accuracy: {test_accuracy}")

**5. AntColony Optimization using SoftSign Activation function**

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
# Loading MNIST data set
mnist = tf.keras.datasets.mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
# Normalizing the pixel values to the range [0, 1]
X_train, X_test = X_train / 255.0, X_test / 255.0
# Converting labels to categorical
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
# Converting labels to categorical
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
# CNN architecture
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', activation='softsign', input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='softsign'))
model.add(Dense(10, activation='softmax'))
# Compiling the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# Define the objective function for Ant Colony Optimization
def objective_function(params):
    learning_rate = params[0]
    batch_size = int(params[1])
    epochs = int(params[2])

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    history = model.fit(X_train, y_train, validation_data=(X_test, y_test),
                        batch_size=batch_size, epochs=epochs, verbose=0)

    train_loss = history.history['loss'][-1]
    train_accuracy = history.history['accuracy'][-1]

    return train_loss + 1 - train_accuracy  # ACO minimizes the objective function, so maximize accuracy.

# Define the search space for ACO
param_boundaries = [
    (1e-6, 1e-2),  # Learning rate
    (32, 128),     # Batch size
    (5, 50)        # Epochs
]
# Number of ants in the colony
num_ants = 10
# Number of iterations
num_iterations = 50
# Evaporation rate
evaporation_rate = 0.5
# Pheromone decay rate
pheromone_decay = 0.1
class AntColonyOptimizer:
    def __init__(self, num_ants, objective_function, param_boundaries, evaporation_rate, pheromone_decay):
        self.num_ants = num_ants
        self.objective_function = objective_function
        self.param_boundaries = param_boundaries
        self.evaporation_rate = evaporation_rate
        self.pheromone_decay = pheromone_decay
        self.pheromone_levels = np.ones(len(param_boundaries[0]))

    def optimize(self, num_iterations):
        # Initialization
        best_params = None
        best_fitness = float('inf')

        for iteration in range(num_iterations):
            for ant in range(self.num_ants):
                params = [self._get_ant_param(boundaries) for boundaries in self.param_boundaries]
                fitness = self.objective_function(params)

                # Update global best
                if fitness < best_fitness:
                    best_params = params
                    best_fitness = fitness

                # Update pheromone levels
                self.pheromone_levels *= (1 - self.pheromone_decay)
                self.pheromone_levels += (1 / (fitness + 1))  # Fitness should be positive, avoid division by zero

            # Evaporation
            self.pheromone_levels *= self.evaporation_rate

        return best_params

    def _get_ant_param(self, boundaries):
        probabilities = self.pheromone_levels / np.sum(self.pheromone_levels)
        selected_index = np.random.choice(len(boundaries), p=probabilities)
        return np.random.uniform(*boundaries[selected_index])
# Creating an Ant Colony Optimizer object
aco = AntColonyOptimizer(num_ants, objective_function, param_boundaries, evaporation_rate, pheromone_decay)
# Finding the optimal parameters
optimal_params = aco.optimize(num_iterations)
# Displaying the best parameters found
print(f"Best parameters found by ACO: {optimal_params}")

# Compiling the model with the best parameters
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=optimal_params[0]),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Training the model with the best parameters
history = model.fit(X_train, y_train, batch_size=int(optimal_params[1]), epochs=int(optimal_params[2]), verbose=1)

# Evaluating the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test loss: {test_loss}, Test accuracy: {test_accuracy}")

**6. AntColony Optimization using Softplus Activation function**

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
# Loading MNIST data set
mnist = tf.keras.datasets.mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
# Normalizing the pixel values to the range [0, 1]
X_train, X_test = X_train / 255.0, X_test / 255.0
# Converting labels to categorical
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
# Converting labels to categorical
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
# CNN architecture
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', activation='softplus', input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='softplus'))
model.add(Dense(10, activation='softmax'))
# Compiling the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# Define the objective function for Ant Colony Optimization
def objective_function(params):
    learning_rate = params[0]
    batch_size = int(params[1])
    epochs = int(params[2])

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    history = model.fit(X_train, y_train, validation_data=(X_test, y_test),
                        batch_size=batch_size, epochs=epochs, verbose=0)

    train_loss = history.history['loss'][-1]
    train_accuracy = history.history['accuracy'][-1]

    return train_loss + 1 - train_accuracy  # ACO minimizes the objective function, so maximize accuracy.

# Define the search space for ACO
param_boundaries = [
    (1e-6, 1e-2),  # Learning rate
    (32, 128),     # Batch size
    (5, 50)        # Epochs
]
# Number of ants in the colony
num_ants = 10
# Number of iterations
num_iterations = 50
# Evaporation rate
evaporation_rate = 0.5
# Pheromone decay rate
pheromone_decay = 0.1
class AntColonyOptimizer:
    def __init__(self, num_ants, objective_function, param_boundaries, evaporation_rate, pheromone_decay):
        self.num_ants = num_ants
        self.objective_function = objective_function
        self.param_boundaries = param_boundaries
        self.evaporation_rate = evaporation_rate
        self.pheromone_decay = pheromone_decay
        self.pheromone_levels = np.ones(len(param_boundaries[0]))

    def optimize(self, num_iterations):
        # Initialization
        best_params = None
        best_fitness = float('inf')

        for iteration in range(num_iterations):
            for ant in range(self.num_ants):
                params = [self._get_ant_param(boundaries) for boundaries in self.param_boundaries]
                fitness = self.objective_function(params)

                # Update global best
                if fitness < best_fitness:
                    best_params = params
                    best_fitness = fitness

                # Update pheromone levels
                self.pheromone_levels *= (1 - self.pheromone_decay)
                self.pheromone_levels += (1 / (fitness + 1))  # Fitness should be positive, avoid division by zero

            # Evaporation
            self.pheromone_levels *= self.evaporation_rate

        return best_params

    def _get_ant_param(self, boundaries):
        probabilities = self.pheromone_levels / np.sum(self.pheromone_levels)
        selected_index = np.random.choice(len(boundaries), p=probabilities)
        return np.random.uniform(*boundaries[selected_index])
# Creating an Ant Colony Optimizer object
aco = AntColonyOptimizer(num_ants, objective_function, param_boundaries, evaporation_rate, pheromone_decay)
# Finding the optimal parameters
optimal_params = aco.optimize(num_iterations)
# Displaying the best parameters found
print(f"Best parameters found by ACO: {optimal_params}")

# Compiling the model with the best parameters
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=optimal_params[0]),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Training the model with the best parameters
history = model.fit(X_train, y_train, batch_size=int(optimal_params[1]), epochs=int(optimal_params[2]), verbose=1)

# Evaluating the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test loss: {test_loss}, Test accuracy: {test_accuracy}")

**7.AntColony Optimization using Swish Activation function**

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
# Loading MNIST data set
mnist = tf.keras.datasets.mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
# Normalizing the pixel values to the range [0, 1]
X_train, X_test = X_train / 255.0, X_test / 255.0
# Converting labels to categorical
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
# Converting labels to categorical
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
# CNN architecture
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', activation='swish', input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='swish'))
model.add(Dense(10, activation='softmax'))
# Compiling the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# Define the objective function for Ant Colony Optimization
def objective_function(params):
    learning_rate = params[0]
    batch_size = int(params[1])
    epochs = int(params[2])

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    history = model.fit(X_train, y_train, validation_data=(X_test, y_test),
                        batch_size=batch_size, epochs=epochs, verbose=0)

    train_loss = history.history['loss'][-1]
    train_accuracy = history.history['accuracy'][-1]

    return train_loss + 1 - train_accuracy  # ACO minimizes the objective function, so maximize accuracy.

# Define the search space for ACO
param_boundaries = [
    (1e-6, 1e-2),  # Learning rate
    (32, 128),     # Batch size
    (5, 50)        # Epochs
]
# Number of ants in the colony
num_ants = 10
# Number of iterations
num_iterations = 50
# Evaporation rate
evaporation_rate = 0.5
# Pheromone decay rate
pheromone_decay = 0.1
class AntColonyOptimizer:
    def __init__(self, num_ants, objective_function, param_boundaries, evaporation_rate, pheromone_decay):
        self.num_ants = num_ants
        self.objective_function = objective_function
        self.param_boundaries = param_boundaries
        self.evaporation_rate = evaporation_rate
        self.pheromone_decay = pheromone_decay
        self.pheromone_levels = np.ones(len(param_boundaries[0]))

    def optimize(self, num_iterations):
        # Initialization
        best_params = None
        best_fitness = float('inf')

        for iteration in range(num_iterations):
            for ant in range(self.num_ants):
                params = [self._get_ant_param(boundaries) for boundaries in self.param_boundaries]
                fitness = self.objective_function(params)

                # Update global best
                if fitness < best_fitness:
                    best_params = params
                    best_fitness = fitness

                # Update pheromone levels
                self.pheromone_levels *= (1 - self.pheromone_decay)
                self.pheromone_levels += (1 / (fitness + 1))  # Fitness should be positive, avoid division by zero

            # Evaporation
            self.pheromone_levels *= self.evaporation_rate

        return best_params

    def _get_ant_param(self, boundaries):
        probabilities = self.pheromone_levels / np.sum(self.pheromone_levels)
        selected_index = np.random.choice(len(boundaries), p=probabilities)
        return np.random.uniform(*boundaries[selected_index])
# Creating an Ant Colony Optimizer object
aco = AntColonyOptimizer(num_ants, objective_function, param_boundaries, evaporation_rate, pheromone_decay)
# Finding the optimal parameters
optimal_params = aco.optimize(num_iterations)
# Displaying the best parameters found
print(f"Best parameters found by ACO: {optimal_params}")

# Compiling the model with the best parameters
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=optimal_params[0]),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Training the model with the best parameters
history = model.fit(X_train, y_train, batch_size=int(optimal_params[1]), epochs=int(optimal_params[2]), verbose=1)

# Evaluating the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test loss: {test_loss}, Test accuracy: {test_accuracy}")

**8. AntColony Optimization using mish Activation function**

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
# Loading MNIST data set
mnist = tf.keras.datasets.mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
# Normalizing the pixel values to the range [0, 1]
X_train, X_test = X_train / 255.0, X_test / 255.0
# Converting labels to categorical
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
# Converting labels to categorical
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
# CNN architecture
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', activation='mish', input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='mish'))
model.add(Dense(10, activation='softmax'))
# Compiling the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# Define the objective function for Ant Colony Optimization
def objective_function(params):
    learning_rate = params[0]
    batch_size = int(params[1])
    epochs = int(params[2])

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    history = model.fit(X_train, y_train, validation_data=(X_test, y_test),
                        batch_size=batch_size, epochs=epochs, verbose=0)

    train_loss = history.history['loss'][-1]
    train_accuracy = history.history['accuracy'][-1]

    return train_loss + 1 - train_accuracy  # ACO minimizes the objective function, so maximize accuracy.

# Define the search space for ACO
param_boundaries = [
    (1e-6, 1e-2),  # Learning rate
    (32, 128),     # Batch size
    (5, 50)        # Epochs
]
# Number of ants in the colony
num_ants = 10
# Number of iterations
num_iterations = 50
# Evaporation rate
evaporation_rate = 0.5
# Pheromone decay rate
pheromone_decay = 0.1
class AntColonyOptimizer:
    def __init__(self, num_ants, objective_function, param_boundaries, evaporation_rate, pheromone_decay):
        self.num_ants = num_ants
        self.objective_function = objective_function
        self.param_boundaries = param_boundaries
        self.evaporation_rate = evaporation_rate
        self.pheromone_decay = pheromone_decay
        self.pheromone_levels = np.ones(len(param_boundaries[0]))

    def optimize(self, num_iterations):
        # Initialization
        best_params = None
        best_fitness = float('inf')

        for iteration in range(num_iterations):
            for ant in range(self.num_ants):
                params = [self._get_ant_param(boundaries) for boundaries in self.param_boundaries]
                fitness = self.objective_function(params)

                # Update global best
                if fitness < best_fitness:
                    best_params = params
                    best_fitness = fitness

                # Update pheromone levels
                self.pheromone_levels *= (1 - self.pheromone_decay)
                self.pheromone_levels += (1 / (fitness + 1))  # Fitness should be positive, avoid division by zero

            # Evaporation
            self.pheromone_levels *= self.evaporation_rate

        return best_params

    def _get_ant_param(self, boundaries):
        probabilities = self.pheromone_levels / np.sum(self.pheromone_levels)
        selected_index = np.random.choice(len(boundaries), p=probabilities)
        return np.random.uniform(*boundaries[selected_index])
# Creating an Ant Colony Optimizer object
aco = AntColonyOptimizer(num_ants, objective_function, param_boundaries, evaporation_rate, pheromone_decay)
# Finding the optimal parameters
optimal_params = aco.optimize(num_iterations)
# Displaying the best parameters found
print(f"Best parameters found by ACO: {optimal_params}")

# Compiling the model with the best parameters
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=optimal_params[0]),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Training the model with the best parameters
history = model.fit(X_train, y_train, batch_size=int(optimal_params[1]), epochs=int(optimal_params[2]), verbose=1)

# Evaluating the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test loss: {test_loss}, Test accuracy: {test_accuracy}")

**9. AntColony Optimization using HardSigmoid Activation function**

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
# Loading MNIST data set
mnist = tf.keras.datasets.mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
# Normalizing the pixel values to the range [0, 1]
X_train, X_test = X_train / 255.0, X_test / 255.0
# Converting labels to categorical
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
# Converting labels to categorical
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
# Define the Hard Sigmoid Activation Layer
class HardSigmoidActivation(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(HardSigmoidActivation, self).__init__(**kwargs)

    def call(self, inputs):
        return tf.keras.backend.hard_sigmoid(inputs)

def create_cnn_model(activation_function):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same', activation=activation_function, input_shape=(28, 28, 1)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation=activation_function))
    model.add(Dense(10, activation='softmax'))
    return model
    # Compiling the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# Define the objective function for Ant Colony Optimization
def objective_function(params):
    learning_rate = params[0]
    batch_size = int(params[1])
    epochs = int(params[2])

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    history = model.fit(X_train, y_train, validation_data=(X_test, y_test),
                        batch_size=batch_size, epochs=epochs, verbose=0)

    train_loss = history.history['loss'][-1]
    train_accuracy = history.history['accuracy'][-1]

    return train_loss + 1 - train_accuracy  # ACO minimizes the objective function, so maximize accuracy.

# Define the search space for ACO
param_boundaries = [
    (1e-6, 1e-2),  # Learning rate
    (32, 128),     # Batch size
    (5, 50)        # Epochs
]
# Number of ants in the colony
num_ants = 10
# Number of iterations
num_iterations = 50
# Evaporation rate
evaporation_rate = 0.5
# Pheromone decay rate
pheromone_decay = 0.1
class AntColonyOptimizer:
    def __init__(self, num_ants, objective_function, param_boundaries, evaporation_rate, pheromone_decay):
        self.num_ants = num_ants
        self.objective_function = objective_function
        self.param_boundaries = param_boundaries
        self.evaporation_rate = evaporation_rate
        self.pheromone_decay = pheromone_decay
        self.pheromone_levels = np.ones(len(param_boundaries[0]))

    def optimize(self, num_iterations):
        # Initialization
        best_params = None
        best_fitness = float('inf')

        for iteration in range(num_iterations):
            for ant in range(self.num_ants):
                params = [self._get_ant_param(boundaries) for boundaries in self.param_boundaries]
                fitness = self.objective_function(params)

                # Update global best
                if fitness < best_fitness:
                    best_params = params
                    best_fitness = fitness

                # Update pheromone levels
                self.pheromone_levels *= (1 - self.pheromone_decay)
                self.pheromone_levels += (1 / (fitness + 1))  # Fitness should be positive, avoid division by zero

            # Evaporation
            self.pheromone_levels *= self.evaporation_rate

        return best_params

    def _get_ant_param(self, boundaries):
        probabilities = self.pheromone_levels / np.sum(self.pheromone_levels)
        selected_index = np.random.choice(len(boundaries), p=probabilities)
        return np.random.uniform(*boundaries[selected_index])
# Creating an Ant Colony Optimizer object
aco = AntColonyOptimizer(num_ants, objective_function, param_boundaries, evaporation_rate, pheromone_decay)
# Finding the optimal parameters
optimal_params = aco.optimize(num_iterations)
# Displaying the best parameters found
print(f"Best parameters found by ACO: {optimal_params}")

# Compiling the model with the best parameters
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=optimal_params[0]),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Training the model with the best parameters
history = model.fit(X_train, y_train, batch_size=int(optimal_params[1]), epochs=int(optimal_params[2]), verbose=1)

# Evaluating the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test loss: {test_loss}, Test accuracy: {test_accuracy}")

**10. AntColony Optimization using Selu Activation function**

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
# Loading MNIST data set
mnist = tf.keras.datasets.mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
# Normalizing the pixel values to the range [0, 1]
X_train, X_test = X_train / 255.0, X_test / 255.0
# Converting labels to categorical
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
# Converting labels to categorical
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
# CNN architecture
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', activation='selu', input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='selu'))
model.add(Dense(10, activation='softmax'))
# Compiling the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# Define the objective function for Ant Colony Optimization
def objective_function(params):
    learning_rate = params[0]
    batch_size = int(params[1])
    epochs = int(params[2])

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    history = model.fit(X_train, y_train, validation_data=(X_test, y_test),
                        batch_size=batch_size, epochs=epochs, verbose=0)

    train_loss = history.history['loss'][-1]
    train_accuracy = history.history['accuracy'][-1]

    return train_loss + 1 - train_accuracy  # ACO minimizes the objective function, so maximize accuracy.

# Define the search space for ACO
param_boundaries = [
    (1e-6, 1e-2),  # Learning rate
    (32, 128),     # Batch size
    (5, 50)        # Epochs
]
# Number of ants in the colony
num_ants = 10
# Number of iterations
num_iterations = 50
# Evaporation rate
evaporation_rate = 0.5
# Pheromone decay rate
pheromone_decay = 0.1
class AntColonyOptimizer:
    def __init__(self, num_ants, objective_function, param_boundaries, evaporation_rate, pheromone_decay):
        self.num_ants = num_ants
        self.objective_function = objective_function
        self.param_boundaries = param_boundaries
        self.evaporation_rate = evaporation_rate
        self.pheromone_decay = pheromone_decay
        self.pheromone_levels = np.ones(len(param_boundaries[0]))

    def optimize(self, num_iterations):
        # Initialization
        best_params = None
        best_fitness = float('inf')

        for iteration in range(num_iterations):
            for ant in range(self.num_ants):
                params = [self._get_ant_param(boundaries) for boundaries in self.param_boundaries]
                fitness = self.objective_function(params)

                # Update global best
                if fitness < best_fitness:
                    best_params = params
                    best_fitness = fitness

                # Update pheromone levels
                self.pheromone_levels *= (1 - self.pheromone_decay)
                self.pheromone_levels += (1 / (fitness + 1))  # Fitness should be positive, avoid division by zero

            # Evaporation
            self.pheromone_levels *= self.evaporation_rate

        return best_params

    def _get_ant_param(self, boundaries):
        probabilities = self.pheromone_levels / np.sum(self.pheromone_levels)
        selected_index = np.random.choice(len(boundaries), p=probabilities)
        return np.random.uniform(*boundaries[selected_index])
# Creating an Ant Colony Optimizer object
aco = AntColonyOptimizer(num_ants, objective_function, param_boundaries, evaporation_rate, pheromone_decay)
# Finding the optimal parameters
optimal_params = aco.optimize(num_iterations)
# Displaying the best parameters found
print(f"Best parameters found by ACO: {optimal_params}")

# Compiling the model with the best parameters
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=optimal_params[0]),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Training the model with the best parameters
history = model.fit(X_train, y_train, batch_size=int(optimal_params[1]), epochs=int(optimal_params[2]), verbose=1)

# Evaluating the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test loss: {test_loss}, Test accuracy: {test_accuracy}")

**11. AntColony Optimization using elu Activation function**

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
# Loading MNIST data set
mnist = tf.keras.datasets.mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
# Normalizing the pixel values to the range [0, 1]
X_train, X_test = X_train / 255.0, X_test / 255.0
# Converting labels to categorical
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
# Converting labels to categorical
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
# CNN architecture
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', activation='elu', input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='elu'))
model.add(Dense(10, activation='softmax'))
# Compiling the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# Define the objective function for Ant Colony Optimization
def objective_function(params):
    learning_rate = params[0]
    batch_size = int(params[1])
    epochs = int(params[2])

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    history = model.fit(X_train, y_train, validation_data=(X_test, y_test),
                        batch_size=batch_size, epochs=epochs, verbose=0)

    train_loss = history.history['loss'][-1]
    train_accuracy = history.history['accuracy'][-1]

    return train_loss + 1 - train_accuracy  # ACO minimizes the objective function, so maximize accuracy.

# Define the search space for ACO
param_boundaries = [
    (1e-6, 1e-2),  # Learning rate
    (32, 128),     # Batch size
    (5, 50)        # Epochs
]
# Number of ants in the colony
num_ants = 10
# Number of iterations
num_iterations = 50
# Evaporation rate
evaporation_rate = 0.5
# Pheromone decay rate
pheromone_decay = 0.1
class AntColonyOptimizer:
    def __init__(self, num_ants, objective_function, param_boundaries, evaporation_rate, pheromone_decay):
        self.num_ants = num_ants
        self.objective_function = objective_function
        self.param_boundaries = param_boundaries
        self.evaporation_rate = evaporation_rate
        self.pheromone_decay = pheromone_decay
        self.pheromone_levels = np.ones(len(param_boundaries[0]))

    def optimize(self, num_iterations):
        # Initialization
        best_params = None
        best_fitness = float('inf')

        for iteration in range(num_iterations):
            for ant in range(self.num_ants):
                params = [self._get_ant_param(boundaries) for boundaries in self.param_boundaries]
                fitness = self.objective_function(params)

                # Update global best
                if fitness < best_fitness:
                    best_params = params
                    best_fitness = fitness

                # Update pheromone levels
                self.pheromone_levels *= (1 - self.pheromone_decay)
                self.pheromone_levels += (1 / (fitness + 1))  # Fitness should be positive, avoid division by zero

            # Evaporation
            self.pheromone_levels *= self.evaporation_rate

        return best_params

    def _get_ant_param(self, boundaries):
        probabilities = self.pheromone_levels / np.sum(self.pheromone_levels)
        selected_index = np.random.choice(len(boundaries), p=probabilities)
        return np.random.uniform(*boundaries[selected_index])
# Creating an Ant Colony Optimizer object
aco = AntColonyOptimizer(num_ants, objective_function, param_boundaries, evaporation_rate, pheromone_decay)
# Finding the optimal parameters
optimal_params = aco.optimize(num_iterations)
# Displaying the best parameters found
print(f"Best parameters found by ACO: {optimal_params}")

# Compiling the model with the best parameters
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=optimal_params[0]),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Training the model with the best parameters
history = model.fit(X_train, y_train, batch_size=int(optimal_params[1]), epochs=int(optimal_params[2]), verbose=1)

# Evaluating the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test loss: {test_loss}, Test accuracy: {test_accuracy}")

**12. AntColony Optimization using PRelu Activation function**

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
# Loading MNIST data set
mnist = tf.keras.datasets.mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
# Normalizing the pixel values to the range [0, 1]
X_train, X_test = X_train / 255.0, X_test / 255.0
# Converting labels to categorical
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
# Converting labels to categorical
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
# CNN architecture
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', activation=' PRelu', input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation=' PRelu'))
model.add(Dense(10, activation='softmax'))
# Compiling the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# Define the objective function for Ant Colony Optimization
def objective_function(params):
    learning_rate = params[0]
    batch_size = int(params[1])
    epochs = int(params[2])

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    history = model.fit(X_train, y_train, validation_data=(X_test, y_test),
                        batch_size=batch_size, epochs=epochs, verbose=0)

    train_loss = history.history['loss'][-1]
    train_accuracy = history.history['accuracy'][-1]

    return train_loss + 1 - train_accuracy  # ACO minimizes the objective function, so maximize accuracy.

# Define the search space for ACO
param_boundaries = [
    (1e-6, 1e-2),  # Learning rate
    (32, 128),     # Batch size
    (5, 50)        # Epochs
]
# Number of ants in the colony
num_ants = 10
# Number of iterations
num_iterations = 50
# Evaporation rate
evaporation_rate = 0.5
# Pheromone decay rate
pheromone_decay = 0.1
class AntColonyOptimizer:
    def __init__(self, num_ants, objective_function, param_boundaries, evaporation_rate, pheromone_decay):
        self.num_ants = num_ants
        self.objective_function = objective_function
        self.param_boundaries = param_boundaries
        self.evaporation_rate = evaporation_rate
        self.pheromone_decay = pheromone_decay
        self.pheromone_levels = np.ones(len(param_boundaries[0]))

    def optimize(self, num_iterations):
        # Initialization
        best_params = None
        best_fitness = float('inf')

        for iteration in range(num_iterations):
            for ant in range(self.num_ants):
                params = [self._get_ant_param(boundaries) for boundaries in self.param_boundaries]
                fitness = self.objective_function(params)

                # Update global best
                if fitness < best_fitness:
                    best_params = params
                    best_fitness = fitness

                # Update pheromone levels
                self.pheromone_levels *= (1 - self.pheromone_decay)
                self.pheromone_levels += (1 / (fitness + 1))  # Fitness should be positive, avoid division by zero

            # Evaporation
            self.pheromone_levels *= self.evaporation_rate

        return best_params

    def _get_ant_param(self, boundaries):
        probabilities = self.pheromone_levels / np.sum(self.pheromone_levels)
        selected_index = np.random.choice(len(boundaries), p=probabilities)
        return np.random.uniform(*boundaries[selected_index])
# Creating an Ant Colony Optimizer object
aco = AntColonyOptimizer(num_ants, objective_function, param_boundaries, evaporation_rate, pheromone_decay)
# Finding the optimal parameters
optimal_params = aco.optimize(num_iterations)
# Displaying the best parameters found
print(f"Best parameters found by ACO: {optimal_params}")

# Compiling the model with the best parameters
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=optimal_params[0]),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Training the model with the best parameters
history = model.fit(X_train, y_train, batch_size=int(optimal_params[1]), epochs=int(optimal_params[2]), verbose=1)

# Evaluating the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test loss: {test_loss}, Test accuracy: {test_accuracy}")

**13. AntColony Optimization using LeakyRelu Activation function**

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
# Loading MNIST data set
mnist = tf.keras.datasets.mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
# Normalizing the pixel values to the range [0, 1]
X_train, X_test = X_train / 255.0, X_test / 255.0
# Converting labels to categorical
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
# Converting labels to categorical
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
# CNN architecture
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', activation='LeakyRelu', input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='LeakyRelu'))
model.add(Dense(10, activation='softmax'))
# Compiling the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# Define the objective function for Ant Colony Optimization
def objective_function(params):
    learning_rate = params[0]
    batch_size = int(params[1])
    epochs = int(params[2])

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    history = model.fit(X_train, y_train, validation_data=(X_test, y_test),
                        batch_size=batch_size, epochs=epochs, verbose=0)

    train_loss = history.history['loss'][-1]
    train_accuracy = history.history['accuracy'][-1]

    return train_loss + 1 - train_accuracy  # ACO minimizes the objective function, so maximize accuracy.

# Define the search space for ACO
param_boundaries = [
    (1e-6, 1e-2),  # Learning rate
    (32, 128),     # Batch size
    (5, 50)        # Epochs
]
# Number of ants in the colony
num_ants = 10
# Number of iterations
num_iterations = 50
# Evaporation rate
evaporation_rate = 0.5
# Pheromone decay rate
pheromone_decay = 0.1
class AntColonyOptimizer:
    def __init__(self, num_ants, objective_function, param_boundaries, evaporation_rate, pheromone_decay):
        self.num_ants = num_ants
        self.objective_function = objective_function
        self.param_boundaries = param_boundaries
        self.evaporation_rate = evaporation_rate
        self.pheromone_decay = pheromone_decay
        self.pheromone_levels = np.ones(len(param_boundaries[0]))

    def optimize(self, num_iterations):
        # Initialization
        best_params = None
        best_fitness = float('inf')

        for iteration in range(num_iterations):
            for ant in range(self.num_ants):
                params = [self._get_ant_param(boundaries) for boundaries in self.param_boundaries]
                fitness = self.objective_function(params)

                # Update global best
                if fitness < best_fitness:
                    best_params = params
                    best_fitness = fitness

                # Update pheromone levels
                self.pheromone_levels *= (1 - self.pheromone_decay)
                self.pheromone_levels += (1 / (fitness + 1))  # Fitness should be positive, avoid division by zero

            # Evaporation
            self.pheromone_levels *= self.evaporation_rate

        return best_params

    def _get_ant_param(self, boundaries):
        probabilities = self.pheromone_levels / np.sum(self.pheromone_levels)
        selected_index = np.random.choice(len(boundaries), p=probabilities)
        return np.random.uniform(*boundaries[selected_index])
# Creating an Ant Colony Optimizer object
aco = AntColonyOptimizer(num_ants, objective_function, param_boundaries, evaporation_rate, pheromone_decay)
# Finding the optimal parameters
optimal_params = aco.optimize(num_iterations)
# Displaying the best parameters found
print(f"Best parameters found by ACO: {optimal_params}")

# Compiling the model with the best parameters
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=optimal_params[0]),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Training the model with the best parameters
history = model.fit(X_train, y_train, batch_size=int(optimal_params[1]), epochs=int(optimal_params[2]), verbose=1)

# Evaluating the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test loss: {test_loss}, Test accuracy: {test_accuracy}")

**14. AntColony Optimization using GELU Activation function**

In [4]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import numpy as np

# Define GELU activation function
def gelu(x):
    cdf = 0.5 * (1.0 + tf.tanh((tf.sqrt(2 / tf.constant(np.pi)) * (x + 0.044715 * x ** 3))))
    return x * cdf

# Loading MNIST data set
mnist = tf.keras.datasets.mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Normalizing the pixel values to the range [0, 1]
X_train, X_test = X_train / 255.0, X_test / 255.0

# Converting labels to categorical
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# CNN architecture with GELU Activation
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', activation=gelu, input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation=gelu))
model.add(Dense(10, activation='softmax'))
# Compiling the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# Define the objective function for Ant Colony Optimization
def objective_function(params):
    learning_rate = params[0]
    batch_size = int(params[1])
    epochs = int(params[2])

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    history = model.fit(X_train, y_train, validation_data=(X_test, y_test),
                        batch_size=batch_size, epochs=epochs, verbose=0)

    train_loss = history.history['loss'][-1]
    train_accuracy = history.history['accuracy'][-1]

    return train_loss + 1 - train_accuracy  # ACO minimizes the objective function, so maximize accuracy.

# Define the search space for ACO
param_boundaries = [
    (1e-6, 1e-2),  # Learning rate
    (32, 128),     # Batch size
    (5, 50)        # Epochs
]
# Number of ants in the colony
num_ants = 10
# Number of iterations
num_iterations = 50
# Evaporation rate
evaporation_rate = 0.5
# Pheromone decay rate
pheromone_decay = 0.1
class AntColonyOptimizer:
    def __init__(self, num_ants, objective_function, param_boundaries, evaporation_rate, pheromone_decay):
        self.num_ants = num_ants
        self.objective_function = objective_function
        self.param_boundaries = param_boundaries
        self.evaporation_rate = evaporation_rate
        self.pheromone_decay = pheromone_decay
        self.pheromone_levels = np.ones(len(param_boundaries[0]))

    def optimize(self, num_iterations):
        # Initialization
        best_params = None
        best_fitness = float('inf')

        for iteration in range(num_iterations):
            for ant in range(self.num_ants):
                params = [self._get_ant_param(boundaries) for boundaries in self.param_boundaries]
                fitness = self.objective_function(params)

                # Update global best
                if fitness < best_fitness:
                    best_params = params
                    best_fitness = fitness

                # Update pheromone levels
                self.pheromone_levels *= (1 - self.pheromone_decay)
                self.pheromone_levels += (1 / (fitness + 1))  # Fitness should be positive, avoid division by zero

            # Evaporation
            self.pheromone_levels *= self.evaporation_rate

        return best_params

    def _get_ant_param(self, boundaries):
        probabilities = self.pheromone_levels / np.sum(self.pheromone_levels)
        selected_index = np.random.choice(len(boundaries), p=probabilities)
        return np.random.uniform(*boundaries[selected_index])
# Creating an Ant Colony Optimizer object
aco = AntColonyOptimizer(num_ants, objective_function, param_boundaries, evaporation_rate, pheromone_decay)
# Finding the optimal parameters
optimal_params = aco.optimize(num_iterations)
# Displaying the best parameters found
print(f"Best parameters found by ACO: {optimal_params}")

# Compiling the model with the best parameters
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=optimal_params[0]),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Training the model with the best parameters
history = model.fit(X_train, y_train, batch_size=int(optimal_params[1]), epochs=int(optimal_params[2]), verbose=1)

# Evaluating the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test loss: {test_loss}, Test accuracy: {test_accuracy}")

